# NX-OS NETCONF Practice Walkthrough

## Explore YANG Models and Review Sample NX-OS NETCONF Python `ncclient` Code
---



### Explore YANG Models

* The NX-OS YANG models are somewhat extensive and there tools available to help you navigate the model trees:
 1. [`pyang`](https://github.com/mbj4668/pyang)
  - Pros - simple, CLI tool, simple installation (via [Python pip](https://pypi.org/project/pyang/)), easy to use.
  - Cons - Can be challenging to navigate through large models.
 2. [YANG Suite](https://developer.cisco.com/yangsuite/)
  - Pros - friendly GUI tool, variety of installation options ([Docker](https://developer.cisco.com/docs/yangsuite/) and [Python pip](https://github.com/CiscoDevNet/yangsuite)), easy to learn and use.
  - Cons - Newer product which has some UI bugs.
 
 **Note:** This walkthrough contains sample NETCONF payloads generated with YANG Suite although the walkthrough does not describe how to use YANG Suite.  The walkthrough will include NETCONF payloads from the files in the [netconf_payloads](/lab/workspaces/auto-I/tree/resources/nexus/netconf/netconf_practice_walkthrough/netconf_payloads) directory.  You may use the NETCONF payload files for reference although you do not need to review them directly.
 
---



###  Interact with NX-OS YANG Modules Using Python **ncclient**

* **ncclient** is a Python NETCONF client.
* Not part of the Python Standard Library - install with **pip**:

```shell
# Already installed in the lab environment
pip install ncclient
```

---
### Import the 'manager' class from the 'ncclient' library & the 'xmltodict' module

In [1]:
from ncclient import manager
import xmltodict

In [2]:
device = {
    'host': '10.10.20.58',
    'port': 830,
    'username': 'admin',
    'password': 'Cisco123',
    'hostkey_verify': 'False',
    'device_params': {
        'name': 'nexus'
    }
}

---
### Get Nexus NETCONF capabilities

In [3]:
with manager.connect(**device) as m:
    caps = m.server_capabilities

header_msg = f'{len(caps)} availabile NETCONF capabilities'
header_border = (f'{"-" * len(header_msg)}')

---
### Print NETCONF capabilities

In [4]:
print(f'\n{header_border}\n'
        f'{header_msg}\n'
        f'{header_border}\n')

for i, c in enumerate(caps):        
    print(f'{i + 1}. {c}')


----------------------------------
38 availabile NETCONF capabilities
----------------------------------

1. urn:ietf:params:netconf:base:1.0
2. urn:ietf:params:netconf:base:1.1
3. urn:ietf:params:netconf:capability:writable-running:1.0
4. urn:ietf:params:netconf:capability:rollback-on-error:1.0
5. urn:ietf:params:netconf:capability:candidate:1.0
6. urn:ietf:params:netconf:capability:validate:1.1
7. urn:ietf:params:netconf:capability:confirmed-commit:1.1
8. urn:ietf:params:netconf:capability:notification:1.0
9. urn:ietf:params:netconf:capability:interleave:1.0
10. urn:ietf:params:netconf:capability:with-defaults:1.0?basic-mode=report-all
11. http://cisco.com/ns/yang/cisco-nx-os-device?revision=2020-07-20&module=Cisco-NX-OS-device
12. http://openconfig.net/yang/acl?revision=2019-11-27&module=openconfig-acl&deviations=cisco-nx-openconfig-acl-deviations
13. http://openconfig.net/yang/bfd?revision=2019-10-25&module=openconfig-bfd&deviations=cisco-nx-openconfig-bfd-deviations
14. http://op

---
### Set the XML payload base path

In [5]:
payload_base_path = 'code/1_netconf_mdp/4_yang_exp_output/'
native = '1_native/'
openconfig = '2_openconfig/'

---
### Configure dictionary of Native & OpenConfig YANG XML payloads

In [6]:
payloads = {
    'Native - Get Serial Number': '1_serial_num_payload.xml',
    'Native - Get Ethernet1/1 Oper': '2_get_eth1_1_payload.xml',
    'Native - Get Ethernet1/1 Config': '3_get_config_eth1_1_payload.xml',
    'Native - Configure Ethernet1/2': '4_edit_config_eth1_2_payload.xml',
    'Native - Get Ethernet 1/2 Config': '5_get_config_eth1_2_payload.xml',
    'Native - Configure Web Server ACL': '6_native_web_server_acl_payload.xml',
    'Native - Copy Run Start': '7_copy_run_start_payload.xml',
    'OpenConfig - Configure Web Server ACL Seq. 10': '1_oc_create_seq_10_payload.xml',
    'OpenConfig - Configure Web Server ACL Seq. 20': '1a_oc_create_seq_20_payload.xml',
    'OpenConfig - Configure Web Server Int. Access Group': '2_oc_apply_e1_2_access_group_payload.xml',
    'OpenConfig - Get Web ACL Config': '3_oc_acl_get_config_payload.xml'
}

for k, v in payloads.items():
    print(f'{k:<50} >> {v}')

Native - Get Serial Number                         >> 1_serial_num_payload.xml
Native - Get Ethernet1/1 Oper                      >> 2_get_eth1_1_payload.xml
Native - Get Ethernet1/1 Config                    >> 3_get_config_eth1_1_payload.xml
Native - Configure Ethernet1/2                     >> 4_edit_config_eth1_2_payload.xml
Native - Get Ethernet 1/2 Config                   >> 5_get_config_eth1_2_payload.xml
Native - Configure Web Server ACL                  >> 6_native_web_server_acl_payload.xml
Native - Copy Run Start                            >> 7_copy_run_start_payload.xml
OpenConfig - Configure Web Server ACL Seq. 10      >> 1_oc_create_seq_10_payload.xml
OpenConfig - Configure Web Server ACL Seq. 20      >> 1a_oc_create_seq_20_payload.xml
OpenConfig - Configure Web Server Int. Access Group >> 2_oc_apply_e1_2_access_group_payload.xml
OpenConfig - Get Web ACL Config                    >> 3_oc_acl_get_config_payload.xml


---
### Setup Nexus **get** serial number NETCONF filter

In [18]:
# Declare YANG Explorer XML payload file name
file_name = f'{payload_base_path}{native}1_serial_num_payload.xml'

# Access YANG Explorer XML payload file & convert to Python dictionary
with open(file_name, 'r') as payload:
    raw_filter = payload.read()
    data = xmltodict.parse(raw_filter,
                           dict_constructor=dict)

# Parse the 'get' key from the dictionary & convert to XML 
nc_data = data['rpc']['get']
filter = xmltodict.unparse(nc_data,
                           full_document=False,
                           pretty=True)

In [19]:
from pprint import pprint as pp
print(raw_filter)
print()
pp(data)
print()
pp(nc_data)
print()
print(filter)

<rpc message-id="101" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
  <get>
    <filter>
      <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
        <serial/>
      </System>
    </filter>
  </get>
</rpc>

{'rpc': {'@message-id': '101',
         '@xmlns': 'urn:ietf:params:xml:ns:netconf:base:1.0',
         'get': {'filter': {'System': {'@xmlns': 'http://cisco.com/ns/yang/cisco-nx-os-device',
                                       'serial': None}}}}}

{'filter': {'System': {'@xmlns': 'http://cisco.com/ns/yang/cisco-nx-os-device',
                       'serial': None}}}

<filter>
	<System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
		<serial></serial>
	</System>
</filter>


---
### Print YANG Explorer XML output

In [20]:
print(raw_filter)

<rpc message-id="101" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
  <get>
    <filter>
      <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
        <serial/>
      </System>
    </filter>
  </get>
</rpc>


---
### Print ncclient filter

In [21]:
print(filter)

<filter>
	<System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
		<serial></serial>
	</System>
</filter>


---
### Send NETCONF **get** request

In [22]:
with manager.connect(**device) as m:
    response = m.get(filter=filter)

---
### Print NETCONF reply

In [23]:
print(response.xml)

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns:nxos="http://www.cisco.com/nxos:1.0" xmlns:if="http://www.cisco.com/nxos:1.0:if_manager" xmlns:nfcli="http://www.cisco.com/nxos:1.0:nfcli" xmlns:vlan_mgr_cli="http://www.cisco.com/nxos:1.0:vlan_mgr_cli" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:1b05dc8e-c162-414d-bb4d-4de101e646eb">
    <data>
        <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
            <serial>91OMZP37WBR</serial>
        </System>
    </data>
</rpc-reply>



---
---
### Setup Ethernet1/1 **get** operational data NETCONF filter

In [24]:
# Declare YANG Explorer XML payload file name
file_name = f'{payload_base_path}{native}2_get_eth1_1_payload.xml'

# Access YANG Explorer XML payload file & convert to Python dictionary
with open(file_name, 'r') as payload:
    raw_filter = payload.read()
    data = xmltodict.parse(raw_filter,
                           dict_constructor=dict)

# Parse the 'get' key from the dictionary & convert to XML 
nc_data = data['rpc']['get']
filter = xmltodict.unparse(nc_data,
                           full_document=False,
                           pretty=True)

---
### Print YANG Explorer XML output

In [25]:
print(raw_filter)

<rpc message-id="101" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
  <get>
    <filter>
      <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
        <intf-items>
          <phys-items>
            <PhysIf-list>
              <id>eth1/1</id>
            </PhysIf-list>
          </phys-items>
        </intf-items>
      </System>
    </filter>
  </get>
</rpc>


---
### Print ncclient filter

In [30]:
manager.connect?

Signature: manager.connect(*args, **kwds)
Docstring: <no docstring>
File:      /usr/local/lib/python3.8/site-packages/ncclient/manager.py
Type:      function


In [35]:
device = {
    'host': '10.10.20.58',
    'port': 830,
    'username': 'admin',
    'password': 'Cisco123',
    'hostkey_verify': False,
    'device_params': {
        'name': 'nexus'
    }
}

In [39]:
dir(conn)

['HUGE_TREE_DEFAULT',
 '_Manager__set_async_mode',
 '_Manager__set_raise_mode',
 '_Manager__set_timeout',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__enter__',
 '__eq__',
 '__exit__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_async_mode',
 '_device_handler',
 '_huge_tree',
 '_raise_mode',
 '_session',
 '_timeout',
 '_vendor_operations',
 'async_mode',
 'channel_id',
 'channel_name',
 'client_capabilities',
 'connected',
 'execute',
 'huge_tree',
 'locked',
 'raise_mode',
 'rpc',
 'scp',
 'server_capabilities',
 'session',
 'session_id',
 'take_notification',
 'timeout']

In [41]:
with manager.connect(**device) as conn:
    r = conn.get(filter=filter)

In [44]:
pp(xmltodict.parse(r.xml, dict_constructor=dict))

{'rpc-reply': {'@message-id': 'urn:uuid:6c9eea87-ef56-4776-aa33-cd48c7c425d9',
               '@xmlns': 'urn:ietf:params:xml:ns:netconf:base:1.0',
               '@xmlns:if': 'http://www.cisco.com/nxos:1.0:if_manager',
               '@xmlns:nfcli': 'http://www.cisco.com/nxos:1.0:nfcli',
               '@xmlns:nxos': 'http://www.cisco.com/nxos:1.0',
               '@xmlns:vlan_mgr_cli': 'http://www.cisco.com/nxos:1.0:vlan_mgr_cli',
               'data': {'System': {'@xmlns': 'http://cisco.com/ns/yang/cisco-nx-os-device',
                                   'intf-items': {'phys-items': {'PhysIf-list': {'FECMode': 'auto',
                                                                                 'accessVlan': 'vlan-1',
                                                                                 'adminSt': 'down',
                                                                                 'aggrmbrif-items': {'bdlPortNum': '0',
                                               

In [26]:
print(filter)

<filter>
	<System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
		<intf-items>
			<phys-items>
				<PhysIf-list>
					<id>eth1/1</id>
				</PhysIf-list>
			</phys-items>
		</intf-items>
	</System>
</filter>


---
### Send NETCONF **get** request

In [ ]:
with manager.connect(**device) as m:
    response = m.get(filter=filter)

---
### Print NETCONF reply

In [ ]:
print(response.xml)

---
---
### Setup Ethernet1/1 **get-config** NETCONF filter

In [45]:
# Declare YANG Explorer XML payload file name
file_name = f'{payload_base_path}{native}3_get_config_eth1_1_payload.xml'

# Access YANG Explorer XML payload file & convert to Python dictionary
with open(file_name, 'r') as payload:
    raw_filter = payload.read()
    data = xmltodict.parse(raw_filter,
                           dict_constructor=dict)

# Parse the 'get-config' key from the dictionary & convert to XML 
nc_data = data['rpc']['get-config']

# Remove the 'source' item
del nc_data['source']
filter = xmltodict.unparse(nc_data,
                           full_document=False,
                           pretty=True)

---
### Print YANG Explorer XML output

In [46]:
print(raw_filter)

<rpc message-id="101" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
  <get-config>
    <source>
      <running/>
    </source>
    <filter>
      <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
        <ipv4-items>
          <inst-items>
            <dom-items>
              <Dom-list>
                <if-items>
                  <If-list>
                    <id>eth1/1</id>
                  </If-list>
                </if-items>
              </Dom-list>
            </dom-items>
          </inst-items>
        </ipv4-items>
      </System>
    </filter>
  </get-config>
</rpc>


---
### Print ncclient filter

In [47]:
print(filter)

<filter>
	<System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
		<ipv4-items>
			<inst-items>
				<dom-items>
					<Dom-list>
						<if-items>
							<If-list>
								<id>eth1/1</id>
							</If-list>
						</if-items>
					</Dom-list>
				</dom-items>
			</inst-items>
		</ipv4-items>
	</System>
</filter>


In [48]:
device = {
    'host': '10.10.20.58',
    'port': 830,
    'username': 'admin',
    'password': 'Cisco123',
    'hostkey_verify': False,
    'device_params': {
        'name': 'nexus'
    }
}

In [53]:
with manager.connect(**device) as conn:
    r = conn.get_config(source='running', filter=filter)

In [57]:
print(dir(r))
pp(xmltodict.parse(r.data_xml, dict_constructor=dict))

['ERROR_CLS', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_data', '_device_handler', '_errors', '_huge_tree', '_parsed', '_parsing_hook', '_raw', '_root', 'data', 'data_ele', 'data_xml', 'error', 'errors', 'ok', 'parse', 'xml']
{'data': {'@xmlns': 'urn:ietf:params:xml:ns:netconf:base:1.0',
          '@xmlns:if': 'http://www.cisco.com/nxos:1.0:if_manager',
          '@xmlns:nfcli': 'http://www.cisco.com/nxos:1.0:nfcli',
          '@xmlns:nxos': 'http://www.cisco.com/nxos:1.0',
          '@xmlns:vlan_mgr_cli': 'http://www.cisco.com/nxos:1.0:vlan_mgr_cli',
          'System': {'@xmlns': 'http://cisco.com/ns/yang/cisco-nx-os-device'}}}


---
### Send NETCONF **get-config** request

In [58]:
with manager.connect(**device) as m:
    response = m.get_config(source="running", filter=filter)

---
### Print NETCONF reply

In [59]:
print(response.xml)

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns:nxos="http://www.cisco.com/nxos:1.0" xmlns:if="http://www.cisco.com/nxos:1.0:if_manager" xmlns:nfcli="http://www.cisco.com/nxos:1.0:nfcli" xmlns:vlan_mgr_cli="http://www.cisco.com/nxos:1.0:vlan_mgr_cli" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:b9107c9e-a80d-4de9-8c70-d63bde426fa9">
    <data>
        <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device"/>
    </data>
</rpc-reply>



---
---
### Setup Ethernet1/2 **edit-config** NETCONF filter

In [60]:
# Declare YANG Explorer XML payload file name
file_name = f'{payload_base_path}{native}4_edit_config_eth1_2_payload.xml'

# Access YANG Explorer XML payload file & convert to Python dictionary
with open(file_name, 'r') as payload:
    raw_config = payload.read()
    data = xmltodict.parse(raw_config,
                           dict_constructor=dict)

# Parse the 'get-config' key from the dictionary & convert to XML 
nc_data = data['rpc']['edit-config']

# Remove the 'target' item
del nc_data['target']
config = xmltodict.unparse(nc_data,
                           full_document=False,
                           pretty=True)

---
### Print YANG Explorer XML output

In [61]:
print(raw_config)

<rpc message-id="101" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
  <edit-config>
    <target>
      <running/>
    </target>
    <config>
      <System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
        <intf-items>
          <phys-items>
            <PhysIf-list>
              <id>eth1/2</id>
              <adminSt>up</adminSt>
              <layer>Layer3</layer>
              <userCfgdFlags>admin_layer,admin_state</userCfgdFlags>
            </PhysIf-list>
          </phys-items>
        </intf-items>
        <ipv4-items>
          <inst-items>
            <dom-items>
              <Dom-list>
                <name>default</name>
                <if-items>
                  <If-list>
                    <id>eth1/2</id>
                    <addr-items>
                      <Addr-list>
                        <addr>192.168.5.2/24</addr>
                        <pref>0</pref>
                        <tag>0</tag>
                      </Addr-list>
                    </ad

---
### Print ncclient filter

In [62]:
print(config)

<config>
	<System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
		<intf-items>
			<phys-items>
				<PhysIf-list>
					<id>eth1/2</id>
					<adminSt>up</adminSt>
					<layer>Layer3</layer>
					<userCfgdFlags>admin_layer,admin_state</userCfgdFlags>
				</PhysIf-list>
			</phys-items>
		</intf-items>
		<ipv4-items>
			<inst-items>
				<dom-items>
					<Dom-list>
						<name>default</name>
						<if-items>
							<If-list>
								<id>eth1/2</id>
								<addr-items>
									<Addr-list>
										<addr>192.168.5.2/24</addr>
										<pref>0</pref>
										<tag>0</tag>
									</Addr-list>
								</addr-items>
							</If-list>
						</if-items>
					</Dom-list>
				</dom-items>
			</inst-items>
		</ipv4-items>
	</System>
</config>


In [64]:
config = """
<config>
	<System xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
		<intf-items>
			<phys-items>
				<PhysIf-list>
					<id>eth1/2</id>
					<adminSt>up</adminSt>
					<layer>Layer3</layer>
					<userCfgdFlags>admin_layer,admin_state</userCfgdFlags>
				</PhysIf-list>
			</phys-items>
		</intf-items>
		<ipv4-items>
			<inst-items>
				<dom-items>
					<Dom-list>
						<name>default</name>
						<if-items>
							<If-list>
								<id>eth1/25</id>
								<addr-items>
									<Addr-list>
										<addr>192.168.5.2/24</addr>
										<pref>0</pref>
										<tag>0</tag>
									</Addr-list>
								</addr-items>
							</If-list>
						</if-items>
					</Dom-list>
				</dom-items>
			</inst-items>
		</ipv4-items>
	</System>
</config>
""".strip()

---
### Pre-check Nexus configuration via **vshell**

```shell
# Verify the pre-configuration interface state
show run interface Ethernet1/2
```

In [67]:
with manager.connect(**device) as m:
    r = m.edit_config(target='candidate', config=config)

In [69]:
print(r)

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns:nxos="http://www.cisco.com/nxos:1.0" xmlns:if="http://www.cisco.com/nxos:1.0:if_manager" xmlns:nfcli="http://www.cisco.com/nxos:1.0:nfcli" xmlns:vlan_mgr_cli="http://www.cisco.com/nxos:1.0:vlan_mgr_cli" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:26616b4b-eebd-4056-beaf-7f4438ffa086">
    <ok/>
</rpc-reply>



---
### Send NETCONF **edit-config** request

In [ ]:
with manager.connect(**device) as m:
    response = m.edit_config(target="candidate", config=config)

---
### Print NETCONF reply

In [ ]:
print(response.xml)

---
### Commit **candidate** datastore to **running** datastore

In [70]:
with manager.connect(**device) as m:
    response = m.commit()

---
### Print NETCONF reply

In [71]:
print(response.xml)

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns:nxos="http://www.cisco.com/nxos:1.0" xmlns:if="http://www.cisco.com/nxos:1.0:if_manager" xmlns:nfcli="http://www.cisco.com/nxos:1.0:nfcli" xmlns:vlan_mgr_cli="http://www.cisco.com/nxos:1.0:vlan_mgr_cli" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:a5c1ba39-a429-4783-bdd8-6e6ec5568b4e">
    <ok/>
</rpc-reply>



---
### Post-check Nexus configuration via **vshell**

```shell
# Verify the post-configuration interface state
show run interface Ethernet1/2
```

---
---
### Setup Ethernet1/2 **get-config** NETCONF filter

In [ ]:
# Declare YANG Explorer XML payload file name
file_name = f'{payload_base_path}{native}5_get_config_eth1_2_payload.xml'

# Access YANG Explorer XML payload file & convert to Python dictionary
with open(file_name, 'r') as payload:
    raw_filter = payload.read()
    data = xmltodict.parse(raw_filter,
                           dict_constructor=dict)

# Parse the 'get-config' key from the dictionary & convert to XML 
nc_data = data['rpc']['get-config']

# Remove the 'source' item
del nc_data['source']
filter = xmltodict.unparse(nc_data,
                           full_document=False,
                           pretty=True)

---
### Print YANG Explorer XML output

In [ ]:
print(raw_filter)

---
### Print ncclient filter

In [ ]:
print(filter)

---
### Send NETCONF **get-config** request

In [ ]:
with manager.connect(**device) as m:
    response = m.get_config(source="running", filter=filter)

---
### Print NETCONF reply

In [ ]:
print(response.xml)

---
---
### Setup OpenConfig web server ACL, sequence 10 **edit-config** NETCONF filter

In [87]:
# Declare YANG Explorer XML payload file name
file_name = f'{payload_base_path}{openconfig}1_oc_create_seq_10_payload.xml'

# Access YANG Explorer XML payload file & convert to Python dictionary
with open(file_name, 'r') as payload:
    raw_config = payload.read()
    data = xmltodict.parse(raw_config,
                           dict_constructor=dict)

# Parse the 'edit-config' key from the dictionary & convert to XML 
nc_data = data['rpc']['edit-config']

# Remove the 'target' item
del nc_data['target']
config = xmltodict.unparse(nc_data,
                           full_document=False,
                           pretty=True)

---
### Print YANG Explorer XML output

In [76]:
print(raw_config)

<rpc message-id="101" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
  <edit-config>
    <target>
      <running/>
    </target>
    <config>
      <acl xmlns="http://openconfig.net/yang/acl">
        <acl-sets>
          <acl-set>
            <name>WEB-SERVER</name>
            <type>ACL_IPV4</type>
            <acl-entries>
              <acl-entry>
                <sequence-id>10</sequence-id>
                <config>
                  <sequence-id>10</sequence-id>
                </config>
                <ipv4>
                  <config>
                    <source-address>192.168.2.0/24</source-address>
                    <destination-address>192.168.5.3/32</destination-address>
                    <protocol>IP_TCP</protocol>
                  </config>
                </ipv4>
                <transport>
                  <config>
                    <destination-port>80</destination-port>
                  </config>
                </transport>
                <actions>
     

---
### Print ncclient filter

In [88]:
print(config)

<config>
	<acl xmlns="http://openconfig.net/yang/acl">
		<acl-sets>
			<acl-set>
				<name>WEB-SERVER</name>
				<type>ACL_IPV4</type>
				<acl-entries>
					<acl-entry>
						<sequence-id>10</sequence-id>
						<config>
							<sequence-id>10</sequence-id>
						</config>
						<ipv4>
							<config>
								<source-address>192.168.2.0/24</source-address>
								<destination-address>192.168.5.3/32</destination-address>
								<protocol>IP_TCP</protocol>
							</config>
						</ipv4>
						<transport>
							<config>
								<destination-port>80</destination-port>
							</config>
						</transport>
						<actions>
							<config>
								<forwarding-action>ACCEPT</forwarding-action>
							</config>
						</actions>
					</acl-entry>
				</acl-entries>
			</acl-set>
		</acl-sets>
	</acl>
</config>


---
### Pre-check Nexus configuration via **vshell**

```shell
# Verify the pre-configuration ACL state
show run | section access-list
```

In [78]:
m = manager.connect(**device)

In [79]:
m.lock(target='candidate')

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns:nxos="http://www.cisco.com/nxos:1.0" xmlns:if="http://www.cisco.com/nxos:1.0:if_manager" xmlns:nfcli="http://www.cisco.com/nxos:1.0:nfcli" xmlns:vlan_mgr_cli="http://www.cisco.com/nxos:1.0:vlan_mgr_cli" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:00eea5f3-713f-42f3-8533-9ef8025b4d42">
    <ok/>
</rpc-reply>

In [80]:
m.edit_config(target='candidate', config=config)

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns:nxos="http://www.cisco.com/nxos:1.0" xmlns:if="http://www.cisco.com/nxos:1.0:if_manager" xmlns:nfcli="http://www.cisco.com/nxos:1.0:nfcli" xmlns:vlan_mgr_cli="http://www.cisco.com/nxos:1.0:vlan_mgr_cli" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:7691478f-fccd-4cd9-a8a9-24623a5ca5be">
    <ok/>
</rpc-reply>

In [81]:
m.unlock()

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns:nxos="http://www.cisco.com/nxos:1.0" xmlns:if="http://www.cisco.com/nxos:1.0:if_manager" xmlns:nfcli="http://www.cisco.com/nxos:1.0:nfcli" xmlns:vlan_mgr_cli="http://www.cisco.com/nxos:1.0:vlan_mgr_cli" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:108a4f22-f945-4f77-b185-3a5bcfb935b1">
    <ok/>
</rpc-reply>

In [82]:
m.commit()

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns:nxos="http://www.cisco.com/nxos:1.0" xmlns:if="http://www.cisco.com/nxos:1.0:if_manager" xmlns:nfcli="http://www.cisco.com/nxos:1.0:nfcli" xmlns:vlan_mgr_cli="http://www.cisco.com/nxos:1.0:vlan_mgr_cli" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:1b73d700-7a0f-4442-9252-fd092cc8d483">
    <ok/>
</rpc-reply>

---
### Lock the **candidate** datastore and send the **edit-config** NETCONF request

In [89]:
with manager.connect(**device) as m:
    m.lock()
    response = m.edit_config(target="candidate", config=config)
    m.unlock()

---
### Print NETCONF reply

In [90]:
print(response.xml)

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns:nxos="http://www.cisco.com/nxos:1.0" xmlns:if="http://www.cisco.com/nxos:1.0:if_manager" xmlns:nfcli="http://www.cisco.com/nxos:1.0:nfcli" xmlns:vlan_mgr_cli="http://www.cisco.com/nxos:1.0:vlan_mgr_cli" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:0eea4872-7cc1-47ce-b854-ddabbd6dc263">
    <ok/>
</rpc-reply>



---
### Commit the **candidate** datastore to the **running** datastore

In [91]:
with manager.connect(**device) as m:
    response = m.commit()

---
### Print NETCONF reply

In [92]:
print(response.xml)

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns:nxos="http://www.cisco.com/nxos:1.0" xmlns:if="http://www.cisco.com/nxos:1.0:if_manager" xmlns:nfcli="http://www.cisco.com/nxos:1.0:nfcli" xmlns:vlan_mgr_cli="http://www.cisco.com/nxos:1.0:vlan_mgr_cli" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:4ae88bf3-991f-47ca-a7d8-8d0157548a1f">
    <ok/>
</rpc-reply>



---
### Post-check Nexus configuration via **vshell**

```shell
# Verify the post-configuration ACL state
show run | section access-list
```

---
---
### Setup OpenConfig web server ACL, sequence 20 **edit-config** NETCONF filter

In [ ]:
# Declare YANG Explorer XML payload file name
file_name = f'{payload_base_path}{openconfig}1a_oc_create_seq_20_payload.xml'

# Access YANG Explorer XML payload file & convert to Python dictionary
with open(file_name, 'r') as payload:
    raw_config = payload.read()
    data = xmltodict.parse(raw_config,
                           dict_constructor=dict)

# Parse the 'edit-config' key from the dictionary & convert to XML 
nc_data = data['rpc']['edit-config']

# Remove the 'target' item
del nc_data['target']
config = xmltodict.unparse(nc_data,
                           full_document=False,
                           pretty=True)

---
### Print YANG Explorer XML output

In [ ]:
print(raw_config)

---
### Print ncclient filter

In [ ]:
print(config)

---
### Pre-check Nexus configuration via **vshell**

```shell
# Verify the pre-configuration ACL state
show run | section access-list
```

---
### Lock the **running** datastore and send the **edit-config** NETCONF request directly to the running configuration

In [ ]:
with manager.connect(**device) as m:
    m.lock()
    response = m.edit_config(target="running", config=config)
    m.unlock()

---
### Print NETCONF reply

In [ ]:
print(response.xml)

---
### Post-check Nexus configuration via **vshell**

```shell
# Verify the post-configuration ACL state
show run | section access-list
```

---
---
### Setup OpenConfig web server ACL, Ethernet1/2 access group outbound **edit-config** NETCONF filter

In [ ]:
# Declare YANG Explorer XML payload file name
file_name = f'{payload_base_path}{openconfig}2_oc_apply_e1_2_access_group_payload.xml'

# Access YANG Explorer XML payload file & convert to Python dictionary
with open(file_name, 'r') as payload:
    raw_config = payload.read()
    data = xmltodict.parse(raw_config,
                           dict_constructor=dict)

# Parse the 'edit-config' key from the dictionary & convert to XML 
nc_data = data['rpc']['edit-config']

# Remove the 'target' item
del nc_data['target']
config = xmltodict.unparse(nc_data,
                           full_document=False,
                           pretty=True)

---
### Print YANG Explorer XML output

In [ ]:
print(raw_config)

---
### Print ncclient filter

In [ ]:
print(config)

---
### Pre-check Nexus configuration via **vshell**

```shell
# Verify the pre-configuration interface state
show run interface Ethernet1/2
```

---
### Send the NETCONF request directly to the **running** datastore, without lock

In [ ]:
with manager.connect(**device) as m:
    response = m.edit_config(target="running", config=config)

---
### Print NETCONF reply

In [ ]:
print(response.xml)

---
### Post-check Nexus configuration via **vshell**

```shell
# Verify the post-configuration interface state
show run interface Ethernet1/2
```

---
### Secondary post-check Nexus configuration via **Terminal**

```shell
# Verify ICMP & TCP connectivity to the web server (192.168.5.3)
ping -c 4 192.168.5.3

curl -i http://192.168.5.3
```

---
---
### Setup OpenConfig web server ACL **get** NETCONF filter

In [ ]:
# Declare YANG Explorer XML payload file name
file_name = f'{payload_base_path}{openconfig}3_oc_acl_get_config_payload.xml'

# Access YANG Explorer XML payload file & convert to Python dictionary
with open(file_name, 'r') as payload:
    raw_filter = payload.read()
    data = xmltodict.parse(raw_filter,
                           dict_constructor=dict)

# Parse the 'get-config' key from the dictionary & convert to XML 
nc_data = data['rpc']['get-config']

# Remove the 'source' item
del nc_data['source']
filter = xmltodict.unparse(nc_data,
                           full_document=False,
                           pretty=True)

---
### Print YANG Explorer XML output

In [ ]:
print(raw_filter)

---
### Print ncclient filter

In [ ]:
print(filter)

---
### Send NETCONF **get** request

In [ ]:
with manager.connect(**device) as m:
    response = m.get(filter=filter)

---
### Print NETCONF reply

In [ ]:
print(response.xml)

---
---
### Setup copy of the running **datastore** to the startup **datastore** NETCONF payload

In [93]:
# Declare YANG Explorer XML payload file name
file_name = f'{payload_base_path}{native}7_copy_run_start_payload.xml'

# Access YANG Explorer XML payload file & convert to Python dictionary
with open(file_name, 'r') as payload:
    raw_payload = payload.read()
    data = xmltodict.parse(raw_payload,
                           dict_constructor=dict)

# Parse the 'rpc' key from the dictionary & convert to XML 
nc_data = data['rpc']

# Remove the '@message-id' and @xmlns items
del nc_data['@message-id']
del nc_data['@xmlns']
payload = xmltodict.unparse(nc_data,
                           full_document=False,
                           pretty=True)

---
### Print YANG Explorer XML output

In [94]:
print(raw_payload)

<rpc message-id="101" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0">
  <copy_running_config_src xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
    <startup-config/>
  </copy_running_config_src>
</rpc>


---
### Print ncclient payload

In [95]:
print(payload)

<copy_running_config_src xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
	<startup-config></startup-config>
</copy_running_config_src>


---
### Pre-check Nexus **running** vs **startu**p configuration via **vshell**

```shell
# Verify the pre-copy interface & ACL state
show run interface Ethernet1/2

show run | sec access-list
```

---
### Send NETCONF **dispatch** request

In [96]:
# Parse the root node from the XML payload
from lxml.etree import fromstring
save_rpc = fromstring(payload)

# Alternate payload format
"""
save_rpc = '''
<copy xmlns="http://cisco.com/ns/yang/cisco-nx-os-device">
    <source>running</source>
    <destination>startup</destination>
</copy>
'''
"""

# Send NETCONF request
with manager.connect(**device) as m:
    response = m.dispatch(save_rpc)

---
### Print NETCONF reply

In [97]:
print(response.xml)

<?xml version="1.0" encoding="UTF-8"?>
<rpc-reply xmlns:nxos="http://www.cisco.com/nxos:1.0" xmlns:if="http://www.cisco.com/nxos:1.0:if_manager" xmlns:nfcli="http://www.cisco.com/nxos:1.0:nfcli" xmlns:vlan_mgr_cli="http://www.cisco.com/nxos:1.0:vlan_mgr_cli" xmlns="urn:ietf:params:xml:ns:netconf:base:1.0" message-id="urn:uuid:e54cee2a-8783-4dd7-8d1b-88aa1d7fa6dd">
    <ok/>
</rpc-reply>



---
### Post-check Nexus **running** vs **startu**p configuration via **vshell**

```shell
# Verify the post-copy interface & ACL state
show run interface Ethernet1/2

show run | sec access-list
```

---